In [1]:
import sys
sys.path.append("../scripts/")
from mcl import *

Duplicate key in file WindowsPath('C:/Users/tauyu/anaconda/lib/site-packages/matplotlib/mpl-data/matplotlibrc'), line 257 ('font.family:  IPAexGothic')


<IPython.core.display.Javascript object>

In [2]:
class ResetMcl(Mcl):
    def __init__(self,envmap,init_pose,num,motion_noise_stds={"nn":0.19,"no":0.001,"on":0.13,"oo":0.2},
                distance_dev_rate=0.14,direction_dev=0.05,alpha_threshold=0.001,expansion_rate=0.2):
        super().__init__(envmap,init_pose,num,motion_noise_stds,distance_dev_rate,direction_dev)
        self.alpha_threshold=alpha_threshold
        self.expansion_rate=expansion_rate
        self.counter=0
        
    def random_reset(self):
         for p in self.particles:
                p.pose=np.array([np.random.uniform(-5.0,5,0),np.random.uniform(-5.0,5.0),np.random.uniform(-math.pi,math.pi)]).T
                p.weight=1/len(self.particles)
    
    def sensor_resetting_draw(self,particle,landmark_pos,ell_obs,phi_obs):
             ##パーティクルの位置を決める##
                psi=np.random.uniform(-np.pi,np.pi) #ランドマークからの方角を選ぶ
                ell=norm(loc=ell_obs,scale=(ell_obs*self.distance_dev_rate)**2).rvs() #ランドマークからの距離を選ぶ
                particle.pose[0]=landmark_pos[0]+ell*math.cos(psi)
                particle.pose[1]=landmark_pos[1]+ell*math.sin(psi)
                
            ##パーティクルの向きを決める##
                phi=norm(loc=phi_obs,scale=(self.direction_dev)**2).rvs() #ランドマークが見える向きを決める
                particle.pose[2]=math.atan2(landmark_pos[1]-particle.pose[1],landmark_pos[0]-particle.pose[0])-phi
            
                particle.weight=1.0/len(self.particles) #重みを1/Nに
            
    def sensor_resetting(self,observation):
        nearest_obs = np.argmin([obs[0][0] for obs in observation]) #距離が一番近いランドマークを選択
        values, landmark_id = observation[nearest_obs]
        
        for p in self.particles:
            self.sensor_resetting_draw(p, self.map.landmarks[landmark_id].pos, *values)
            
            
    def expansion_resetting(self):
        for p in self.particles:
            p.pose+=multivariate_normal(cov=np.eye(3)*(self.expansion_rate**2)).rvs()
            p.weight=1.0/len(self.particles)
            
    def observation_update(self,observation):
        for p in self.particles:
            p.observation_update(observation,self.map,self.distance_dev_rate,self.direction_dev)
     
        self.set_ml()
        
        if sum([p.weight for p in self.particles]) < self.alpha_threshold:
            self.counter += 1
            if self.counter < 5:
                self.expansion_resetting() #expansion_resettingに変更
            else:
                self.sensor_resetting(observation)
        else:
            self.counter=0
            self.resampling()

In [3]:
def trial_phantom(animation): ###mclkidnap1test
    time_interval=0.1
    world=World(30,time_interval,debug=not animation)
    
    ##地図を生成して3つランドマークを追加##
    m=Map()
    m.append_landmark(Landmark(-4,2))
    m.append_landmark(Landmark(2,-3))
    m.append_landmark(Landmark(3,3))
    world.append(m)
    
    ##ロボットを作る##
    init_pose=np.array([0,0,0]).T
    robot_pose=np.array([0,0,0]).T
    pf=ResetMcl(m,init_pose,100)
    a=EstimationAgent(time_interval,0.2,10.0/180*math.pi,pf)
    r=Robot(robot_pose,sensor=Camera(m,phantom_prob=0.1),agent=a,color="red")
    world.append(r)
    
    world.draw()
    
    return(r.pose, pf.ml.pose)

In [4]:
trial_phantom(True)

C:\Users\tauyu\anaconda\lib\site-packages\matplotlib\animation.py:889: UserWarning: Animation was deleted without rendering anything. This is most likely not intended. To prevent deletion, assign the Animation to a variable, e.g. `anim`, that exists until you have outputted the Animation using `plt.show()` or `anim.save()`.
  warnings.warn(


<IPython.core.display.Javascript object>

(array([0, 0, 0]), array([0, 0, 0]))

In [ ]:
if __name__=='__main__': ###mclglobal1exec
    ok=0
    for i in range(1000):
        actual, estm=trial_phantom(False)
        diff = math.sqrt((actual[0]-estm[0])**2 + (actual[1]-estm[1])**2)
        print(i, "真値:",actual, "推定値:", estm, "誤差:", diff)
        if diff<=1.0:
            ok += 1
            print(ok)

0 真値: [-0.49515452  0.16364609  5.49431447] 推定値: [-1.00951917 -0.12498112  5.68992454] 誤差: 0.5898106985695521
1
1 真値: [-0.68691509  0.9460358   5.50166663] 推定値: [-0.15796736  0.40207296  5.36605983] 誤差: 0.7587366353481925
2
2 真値: [0.01069708 0.27724856 6.27248801] 推定値: [-0.0865313   0.21370604  6.23403475] 誤差: 0.116150813363124
3
3 真値: [-0.76192167  0.59825833  5.62578181] 推定値: [-0.85259333  0.8080348   5.65346439] 誤差: 0.22853340306471273
4
4 真値: [-0.37313372 -0.25945758  5.75189978] 推定値: [-0.62111272 -1.02653774  5.89001643] 誤差: 0.8061671936113703
5
5 真値: [-0.48388796  0.02704132  5.52555595] 推定値: [0.44803463 0.70798681 5.21105987] 誤差: 1.15419516518629
6 真値: [-1.47844246  0.90594354  4.73621973] 推定値: [-1.2608433   1.60689552  4.79526503] 誤差: 0.7339503190152012
6
7 真値: [-0.02302126 -0.24483667  6.44795267] 推定値: [ 0.20767792 -0.22941923  6.5473648 ] 誤差: 0.23121377322098038
7
8 真値: [-1.71311649  0.94617964  4.82777111] 推定値: [-1.585271    0.59104619  5.07943795] 誤差: 0.3774443486931924
8
9

C:\Users\tauyu\section_advanced_localization\../scripts\ideal_robot.py:30: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(4,4))


20 真値: [-1.17275809  2.11998225  4.26305182] 推定値: [-0.99629591  2.67264083  4.41408622] 誤差: 0.580146883098984
19
21 真値: [-0.82517767  0.45739984  5.4210697 ] 推定値: [-0.67228827  0.96634931  5.28076603] 誤差: 0.5314176603862707
20
22 真値: [-1.21741206  0.85099105  5.42661553] 推定値: [-0.92604222  0.61856426  5.39556153] 誤差: 0.37271785185717854
21
23 真値: [-0.71896825  0.92065168  5.35039372] 推定値: [-0.45175115  2.39049674  5.17697164] 誤差: 1.4939375793372986
24 真値: [ 0.48420049 -0.21012252  6.53357205] 推定値: [ 0.63628097 -0.38095225  6.55863091] 誤差: 0.22871656618824587
22
25 真値: [-1.0874159   0.43803281  5.24267273] 推定値: [-1.67953015  0.50541189  5.35872715] 誤差: 0.5959355836601133
23
26 真値: [-0.39729759  0.10286601  5.71925842] 推定値: [-0.33798818 -1.31559411  6.04241348] 誤差: 1.419699517228097
27 真値: [-0.22235803 -0.04163245  6.35640817] 推定値: [-0.1437909   0.07423215  6.28622477] 誤差: 0.13999070241343992
24
28 真値: [-2.15926823  1.15963402  4.44462661] 推定値: [-1.36066212  2.63263924  4.54275742] 誤差: 1